<a href="https://colab.research.google.com/github/Wang-Kevin900/DataVisualization/blob/main/DataVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup libraries

Donwloading the various library needed to use the program using the console thanks to the symbol "!"

In [124]:
!pip install bokeh
!pip install altair
!pip install plotly
!pip install pandas
!pip install datetime

# Get data and setup

Importing the txt sheet where all the data are (format: date - hour - value) (as long there's new line to indicate that a set of value is done, the program will work no matter how the data is placed in the datasheet, it is fault resistant) and lastly creating all the needed variable

In [125]:
from google.colab import files
from datetime import datetime

# get the data from the text file
file_data = files.upload()

# trasform the data (dictionary) got from the txt file to list, 
# so that in the next FOR cyclefor trasforming the data is easier to read letter per letter
list_of_data_file = list(file_data.values());

# get file name (the key) from the dictionary file_data
list_file_name = list(file_data.keys())

Saving ABT_003_AndamentoConProblemiDiGas ma non critico  BF 32H VASCA BT.txt to ABT_003_AndamentoConProblemiDiGas ma non critico  BF 32H VASCA BT (7).txt
Saving ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI.txt to ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI (7).txt
Saving ABT_001_BT_def_estesi 85M-S VASCA SURGELATI.txt to ABT_001_BT_def_estesi 85M-S VASCA SURGELATI (7).txt
Saving ABT_002_BS 11 VASCA SURGELATI EsempioUtilePerAndamentoRipetitivoPulitoTESTATA.txt to ABT_002_BS 11 VASCA SURGELATI EsempioUtilePerAndamentoRipetitivoPulitoTESTATA (8).txt


Preparing the needed variable to trasform the data read from the datasheet to usable data (in lists)

In [126]:
# variable that indicate the number of symbols that compose a date&time
date_format = 10
time_format = 8

# variable that will store every symbol read from the datasheet
buffer_of_data = ""

# variable that will store the name/title of the various group of data 
# (it has:"No Name" value in case the file is missing a title for the group of data)
element_name = "No Name"

# variable that control if the line that the program is reading, is a data line or group title line
control_name_line = False

# variable that will store any complete data with any of the following format (date, time, float)
single_string_list_of_data = []

# once a single line of data (couple of data) is done being stored, 
# divide them (these 2 list are not really needed in the program, they are here only for code cleaness sake)
list_of_datetime = []
list_of_float = []

# a general list where the complete data (by group) will be stored
data_list = []

# variable that checks if it's the first cycle inside the for loop since it started reading 
# (this is useful for avoiding adding data to the data list when it encounters the first title)
# (since the program add to the data list before each title read)
first_read = True

# divide a single line of data of the datasheet in 3 section, where each section has the following meaning,
# Date, Time, Float; what this variable do, is basically acting as index 
# to indicate wich section we are currently reading in the datasheet
section_index = 0

# variable that keeps track of the file index we are currently reading from the list
index_file = 0

# Trasform Data

Divide the string of data of a single file in 3 section (Date, Time, Float) to easily read the data and do the casting of the type of data.

Then rearrange all the data so that it returns a general list where all the data are stored with the following format: 
- Title; 
-NameOfTheFile; 
- DateTime;
- Value.

while also doing the casting from string to the needed type (for example string to datatime format)

In [127]:
from datetime import datetime

for single_data_file in list_of_data_file:
  # go through all the data that were in the datasheet, and return a general list where are stored all the data (sorted by group)
  # the that reading actually works is that we divide each line of data in 3 section, and each section has different meaning
  # the order of meaning of each section is: data, time, value (this is useful so that we know what we are actually reading)
  for i in single_data_file:
    # 32 is a value ascii that means "Space"
    # 9  is a value ascii that means "Tab"
    # 13 is a value ascii that means "Carriage Return"
    # 10 is a value ascii that means "New Line"
    # Ignore all the "special symbols" (expect 10 since it is the last symbol before the new line is complete, 
    # and it is needed to know when the program is done reading a single line of data), and just get all the "interesting" data
    if i != 32 and i != 9 and i != 13:
      # read a single symbol (convert it from ASCII to string) from string of data (taken from the datasheet) and store it
      buffer_of_data = buffer_of_data + chr(i);

      # if we are currently in the first section (date section)...
      if section_index == 0:
        # if the lenght of the collected data reached the date format, that means that we are done with the date section,
        # also it means that the casting mey be possible (it can fail in case the line read is a "Title" line)
        if len(buffer_of_data) == date_format or control_name_line == True:
          # if the line read is not a "title" line...
          if control_name_line == False:
            # try to do the casting (from string to datetime), and if it fail, it means that we are currently in a "title" line 
            # (since in the first section there could be only a date format or string format (that is the title))
            try:
              single_string_list_of_data.append(datetime.strptime(buffer_of_data, "%d-%m-%Y").date())

              # if the read is successful, that means we are done reading the first data
              first_read = False

              # reset the buffer and pass to the next section
              buffer_of_data = ""
              section_index+=1
            except:
              # if the casting failed, that means we're in the "title" line
              control_name_line = True

          # if we're in the "title" line
          elif control_name_line:
            # wait to gather all the data of the title (we know it's done reading when we encounter the new line symbol)
            if i == 10:
              # if we've done the first reading, it means that we can add the data of the previous group 
              # (this will implicate that outside of the for, we need to add another function to add the data of the last list of data)
              # the program add the data by group each time it reads a title 
              # (the first read variable is useful to ignore the add data during the first reading of the first title)
              if first_read == False:

                data_list.append([element_name, list_file_name[index_file], list_of_datetime.copy(), list_of_float.copy()])

                # clean the 2 lists to prepare them for the upcoming data of other group of data
                list_of_datetime.clear()
                list_of_float.clear()

              # reset all the "support" variable
              element_name = buffer_of_data
              buffer_of_data = ""
              control_name_line = False

        # if the symbol we got from the reading equals 10 (new line), that means that the line we are currently in the file, hasn't any data, so clean the buffer and go on
        elif i == 10:
          buffer_of_data = ""

      # if we are currently in the second section (time section)...
      elif section_index == 1:
        # if the lenght of the collected data reached the time format, that means that we are done with the time section,
        # also it means that the casting is possible
        if len(buffer_of_data) == time_format:
          single_string_list_of_data.append(datetime.strptime(buffer_of_data, "%H:%M:%S").time())

          # reset the buffer and pass to the next section
          buffer_of_data = ""
          section_index+=1

      # if we are currently in the third section (float section)...
      elif section_index == 2:
        # if we detect a 10 (in ASCII means "new line"), that means we're done reading a single line of data
        # and that we are done to also reading a single couple of data
        if i == 10:
          # Add the string to the string of data while also,
          # Converting the value from string to float and replacing the "," in "." so that the casting is possible
          single_string_list_of_data.append(float(buffer_of_data.replace(",", ".")))
          
          # divide the couple of data into 2 different lists to prepare them to be converted into pandas dataframe
          list_of_datetime.append(datetime.combine(single_string_list_of_data[0], single_string_list_of_data[1]))
          list_of_float.append(single_string_list_of_data[2])

          # reset the buffer and the single string of data list and return to the first section to read more data of other lines
          buffer_of_data = ""
          single_string_list_of_data.clear()
          section_index = 0

  # add 1, to indicate that we're going to read the next file (if it exist)
  index_file+=1

# add items for the last time in the list if the last element in the txt file doesn't have \n 
# (the consequence is that the for loop is unable to store the last couple of data)
if single_data_file[-1] != 10 and single_data_file[-1] != 13:
  single_string_list_of_data.append(float(buffer_of_data.replace(",", ".")))
  list_of_datetime.append(datetime.combine(single_string_list_of_data[0], single_string_list_of_data[1]))
  list_of_float.append(single_string_list_of_data[2])

# add the last group of data since the for cycle is unable to do it, because it adds to the list only when it encounters a new "title" 
# (and obviously at the end of a datasheet there can't be a title)
data_list.append([element_name, list_file_name[-1], list_of_datetime.copy(), list_of_float.copy()])

# Choosing and Organizing the data


In [128]:
# a variable that keeps track of the cycle number of the for cycle
i = 0

# Escape Values for highlighting the important data for the user to see
OKBLUE = '\033[34m'
NormalColor = '\033[0m'

# display to the user the possible choices
print("Choose one of the following group of data to visualize (by inserting the number of the group):")
for data in data_list:
  print(str(i)+ ") "+OKBLUE+ data[0] + NormalColor + "File - " + data[1])
  i+=1

# get from the user it's choice, and if it's valid, get the group of data selected
while(True):
  console_input = input()

  if 0 <= int(console_input) and int(console_input) < len(data_list):
    data_list = data_list[int(console_input)].copy()
    break
  else:
    print("Please Insert a valid choice/number")

Choose one of the following group of data to visualize (by inserting the number of the group):
0) Temp.Ripresa
File - ABT_003_AndamentoConProblemiDiGas ma non critico  BF 32H VASCA BT.txt
1) Temp.Regolazione
File - ABT_003_AndamentoConProblemiDiGas ma non critico  BF 32H VASCA BT.txt
2) Temp.Sbrinamento
File - ABT_003_AndamentoConProblemiDiGas ma non critico  BF 32H VASCA BT.txt
3) Sbrinamento1
File - ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI.txt
4) Temperaturaripresa(Sr)
File - ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI.txt
5) Temperaturamandata(Sm)
File - ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI.txt
6) Temperaturasbrinamento(Sd)
File - ABT_004_EsempioDiPeggioramentoUltimogiorno 76 TEST.IRIOS SURGELATI.txt
7) Statodidefrost
File - ABT_001_BT_def_estesi 85M-S VASCA SURGELATI.txt
8) SondaripresaMaster
File - ABT_001_BT_def_estesi 85M-S VASCA SURGELATI.txt
9) SondaripresaSlave
File - ABT_002_BS 11 VASCA SURGELATI 

Convert all the lists (date list and value list) in Pandas DataFrame, then, visualize the list created

In [129]:
import pandas as pd

# convert the 2 lists in a pandas dataframe (order of the list is: Title, Date, Value)
data = pd.DataFrame({'date': data_list[2],
                     'value': data_list[3]})

# print out the datagram
data

,date,value
0,2020-11-08 00:17:06,-13.9
1,2020-11-08 00:37:04,-14.0
2,2020-11-08 00:56:20,-14.0
3,2020-11-08 01:17:05,-14.1
4,2020-11-08 01:37:03,-14.1
...,...,...
940,2020-11-21 03:37:03,-11.1
941,2020-11-21 03:56:23,-12.0
942,2020-11-21 04:16:49,-12.7
943,2020-11-21 04:36:44,-13.2


# Draw Chart

Draw the Chart using the Bokeh library (the reason why i used this library is because it is easily customizable and it supports the callback event, in wich though, you have to write the code in JS; there is a method to convert some data from JS data kernel to Python data kernel by using IPython, but because of google colab enviroment (which do not support the Jupyter Notebook enviroment very well), it is not possible.
Also note that the Google Colab enviroment restrict a lot of interactivity possibilities (for example we are unable to create a bokeh server with the consequences of inability to use curdoc, that is important to add interactive feature to the program).
(a solution is to install and use a local Jupyter Notebook))

In [130]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Button
from bokeh.models.tools import *
from bokeh.models.widgets import DataTable, TableColumn, DateFormatter
from bokeh.layouts import row, column


# Configure the default output state to generate output in notebook cells when show() is called. 
output_notebook()

Create and setup the Main Plot whereas the various data will be displayed and interacted with

In [131]:
# ColumnDataSource is a special object, wich it is easy to share data between multiple plots and widgets
# Also the same ColumnDataSource is used to drive multiple renderers, selections of the data source are also shared.
main_source = ColumnDataSource(data)

# select the desired tool that can be used with the chart
chart_tools="pan, wheel_zoom, reset, hover, poly_select, box_select"
tooltip = HoverTool(
    # specify wich type of info i want to show
    tooltips=[
        ("index", "$index"),
        ("Value", "@value{0.2f}"),
        ("Date", "@date{%F}"),
    ],
    # format the date in datetime type (the default is numerical format)
    formatters={
        '@date': 'datetime'
    },
    # display a tooltip whenever the cursor is vertically in line with a data point
    mode='vline'
)

# add the title to the plot (order of the list: title, date, value) and also
# prepare the chart with the selected options, and add the tooltip too while also activating wheel zoom tool by default
main_chart = figure(title=data_list[0]+" chart", plot_width=1500, plot_height=400, y_axis_label='Values', x_axis_label='Date', x_axis_type='datetime',
           tools=chart_tools, toolbar_location='above', tooltips="", sizing_mode="scale_width")
main_chart.add_tools(tooltip)
main_chart.toolbar.active_scroll = main_chart.select_one(WheelZoomTool)

# draw in the chart both line and circle, so that the obtained effect is that of linked points
symbol1 = main_chart.line(x='date', y='value', line_width=2, source=main_source)
symbol2 = main_chart.circle(x='date', y='value', fill_color="white", size=6, source=main_source)

Create and setup the Table Plot that will display a log that keeps track of all actions (select) done by the user (while also adding extra info)

In [132]:
# create the source for table obj
source_table = ColumnDataSource(data=dict(
    Start_date=[],
    End_date=[],
    Max_value=[],
    Average_value=[],
    Area=[]
))

# specify the each column property of the dataTable obj
columns = [
    TableColumn(field="Start_date", title="Start_date", formatter=DateFormatter(format="%m/%d/%Y - %H:%M:%S")),
    TableColumn(field="End_date", title="End_date", formatter=DateFormatter(format="%m/%d/%Y - %H:%M:%S")),
    TableColumn(field="Max_value", title="Max_value"),
    TableColumn(field="Average_value", title="Average_value"),
    TableColumn(field="Area", title="Area")
]

data_table = DataTable(columns=columns, source=source_table, width=700, height=250)

Create and setup the Bar Plot that will display certain info (Max Value, Avarage Value, Area) whenever the user select some data from the main plot

In [133]:
# create the source for bar plot obj
source_barplot = ColumnDataSource(data=dict(
    name_bar=["Max Value", "Avarage Value", "Area"],
    value=[0, 0, 0]
))
bar_plot = figure(title="General Info", y_range=source_barplot.data["name_bar"], height=300, toolbar_location=None, tools="")
bar_plot.hbar(y="name_bar", right="value", source=source_barplot, width=20)
bar_plot.ygrid.grid_line_color = None

Create and setup a button that has the job to save the log of the data table in a txt file

In [134]:
# create a variable that simulate the \n function, it gets passed as parameter in the save button 
# because in the customJS enviroment, the kernel can't read corretly \n because it execute its function diretly in the code
# instead of passing it as value, so we just pass its value in a variable so that the kernel doesn't read \n directly
break_lines = "\r\n"
save_button = Button(label="Save Data!", button_type="success", height=50)

Specify how the Main Plot will behave when it gets the event that user has selected some of its data (we have to specify by writing in js code); it has to write down the selected data in the table data & bar plot for display purposes

In [135]:
# Execute the JS code when the user select any data in the chart with the box select tool
main_source.selected.js_on_change("indices", CustomJS(args=dict(main_source=main_source, source_table=source_table, source_barplot=source_barplot), code="""
        // get the indices of the selectd data and sort them in ascending order
        var inds = cb_obj.indices
        inds.sort(function(a, b){return a-b})


        // if the array is not empty (if it is empy, it means that no elements has been selected (occurs when the user select no element after having selected some element), 
        // with the consequence of not adding any element)
        if (inds.length != 0){
          // create various variable to store the data from the various sources (ColumnDataSource object)
          var d1 = main_source.data
          var d2 = source_table.data
          var d3 = source_barplot.data


          // get the first and end date (no need to any particular calculus because each data is already in order and unique)
          var start_date = new Date(d1["date"][inds[0]])
          var end_date = new Date(d1["date"][inds[inds.length-1]])


          // get max value (and set it to max 2 decimals)
          var max_value
          for(var i = 0; i < inds.length; i++) {
              if(i == 0) {
                max_value = d1["value"][inds[i]]
              } else {
                if(max_value < d1["value"][inds[i]]){
                  max_value = (d1["value"][inds[i]]).toFixed(2)
                }
              }
          }


          // get average value (and set it to max 2 decimals)
          var avarage_value = 0.00
          for(var i = 0; i < inds.length; i++) {
              avarage_value = avarage_value + d1["value"][inds[i]]
          }
          avarage_value = (avarage_value/inds.length).toFixed(2)


          // calculate the area of the trapezoid (the sum of the two bases (we count the value as minutes) * h (difference between the dates in minutess) / 2)
          var area = 0.00
          for(var i = 0; i < (inds.length-1); i++) {
              var sum_bases = Math.abs(d1["value"][inds[i]]) + Math.abs(d1["value"][inds[i+1]])
              console.log(Math.abs(d1["value"][inds[i]]))
              console.log(Math.abs(d1["value"][inds[i+1]]))
              console.log(sum_bases)
              var height = ((new Date(d1["date"][inds[i+1]])).getTime() - (new Date(d1["date"][inds[i]])).getTime()) / 60000

              area = area + (sum_bases * height) / 2
          }
          // set the number of decimals to 2 
          area = parseFloat(area).toFixed(2);


          // push all the data in the source of the data_table
          d2["Start_date"].push(start_date)
          d2["End_date"].push(end_date)
          d2["Max_value"].push(max_value)
          d2["Average_value"].push(avarage_value)
          d2["Area"].push(area)


          // reset the values of the bar plot and update them (order of values is: Max Value, Avarage Value, Area)
          d3["value"] = []
          d3["value"].push(max_value)
          d3["value"].push(avarage_value)
          d3["value"].push(area)


          // update the Table and Bar plot (this function notify that something has changed, and to immediately update the values and UI)
          source_table.change.emit()
          source_barplot.change.emit()
        }
        """
))

Specify how the Save Button will behave when it gets pressed by the user (we have to specify by writing in js code); it has to get the data from the table data and write it down in a txt file and donwload it

In [136]:
save_button.js_on_click(CustomJS(args=dict(source_table=source_table, break_lines=break_lines), code="""
        // get the data from the source of the table
        var data = source_table.data

        // get the length of the list 
        var number_of_data = data["Start_date"].length

        // order the data
        var start_date_list = data["Start_date"]
        var end_date_list = data["End_date"]
        var max_value_list = data["Max_value"]
        var avarage_value_list = data["Average_value"]
        var area_list = data["Area"]

        // fill the string with the data (while also modifying the data of date format of: hour,minute and seconds to 2 digit string)
        var out = "";
        for (var i = 0; i < number_of_data; i++) {
          out += i+ ")" +break_lines+ 
                 "Start Date:    "  +start_date_list[i]+ "" +break_lines+
                 "End Date:      "  +end_date_list[i]+ "" +break_lines+
                 "Max Value:     "  +max_value_list[i]+ "" +break_lines+
                 "Avarage Value: "  +avarage_value_list[i]+ "" +break_lines+
                 "Area:          "  +area_list[i]+ "" +break_lines+ "" +break_lines
        }


        // write the data in the file
        var file = new Blob([out], {type: 'text/plain'});
        var elem = window.document.createElement('a');
        elem.href = window.URL.createObjectURL(file);
        elem.download = 'selected_data.txt';
        document.body.appendChild(elem);
        elem.click();
        document.body.removeChild(elem);
    """
))

And finally display all the various elements in a certain layout

In [142]:
# visualize the chart in the output
layout = column(main_chart, row(bar_plot,  column(data_table, save_button)))
show(layout)

# Other Library used to visualize data


Test of creating the chart using various libraries...


Draw everything in a chart using the altair library (i haven't used this library since it doesn't have any support for return the data selected)

In [138]:
import altair as alt

brush = alt.selection_interval(encodings=['x'])

color = alt.condition(brush, alt.value('blue'), alt.value('lightgray'))

chart = alt.Chart(data).mark_line(point={
    "filled": False,
    "fill": 'white'
    }).encode(
        y=alt.Y('value', sort=alt.EncodingSortField('value', order='descending')),
        x='date',
        color=color
).add_selection(
    brush
)

chart 

alt.Chart(...)

Draw everything in a chart using the Plotly library (i haven't used this library since it doesn't have any support for return the data selected, an exemption is DASH that is the framework built-on this library, but using this, require a few tweaks)

In [139]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=data.date, y=data.value, mode="markers+lines"))

scatter = fig.data[0]

def update_point(trace, points, selector):
  print("Sleeping")
scatter.on_click(update_point)

fig.show(renderer="colab")

# Machine Learning


In [150]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [154]:
value_list = data_list[3]
defrosting_list = []

for element in value_list:
  if element > 0:
    defrosting_list.append(1)
  elif element <= 0:
    defrosting_list.append(0)

x_train, x_test, y_train, y_test = train_test_split(value_list, defrosting_list, test_size=0.2)



[-15.2, -17.1, -11.3, -16.6, -13.7, -16.2, -15.8, -16.7, -16.1, -8.3, -15.7, -10.9, -15.9, -17.7, -11.4, -16.2, -12.5, -14.4, -12.4, -14.5, -14.5, -16.7, -12.2, 2.6, -12.5, -15.6, -18.5, -16.1, -16.5, -17.1, -6.4, -12.0, -17.6, -16.0, -17.0, -12.1, -13.3, -11.6, -16.3, -7.6, -10.2, -12.4, -15.2, -15.5, -17.8, -16.4, -3.9, -14.8, -8.7, -17.8, -14.4, -17.5, -11.1, -16.1, -12.9, -14.4, -14.8, -17.3, -16.4, -15.6, -12.0, -12.1, -13.2, -15.6, -13.8, -15.7, -8.7, -11.4, -14.4, -13.1, -16.5, -17.6, -2.2, -17.3, -16.0, -11.4, -14.5, -16.9, -16.3, -6.5, -12.6, -16.5, -14.4, -17.4, -14.2, -15.9, -17.8, -11.6, -16.3, -18.5, -12.8, -12.3, -9.9, -15.2, -16.4, -15.5, -5.8, -14.7, -14.6, -13.7, -2.2, -2.7, -14.6, -1.2, -0.8, -16.9, -18.0, -14.4, -15.7, -15.0, -16.0, -11.1, -14.2, -16.8, -16.5, -14.4, -10.1, -17.7, -15.7, -10.2, -14.8, -9.6, -16.1, -16.0, -12.1, -13.6, -16.5, -12.0, -17.0, -16.5, -13.3, -16.6, -11.8, -17.9, -15.8, -13.7, -15.8, -15.8, -17.9, -14.7, -15.7, -14.1, -15.6, -14.6, -7.5, -1